In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd


url = 'https://dhss.delaware.gov/dhss/dph/hsp/Default.aspx?listAll=1&sort=Establishment'
response = requests.get(url)
if response.status_code == 200:
    page_content = response.text


Use requests to get the data from the website into a text format for parsing.


In [2]:
soup = BeautifulSoup(page_content, 'html.parser')
table = soup.find('table')
rows = []
for row in table.findAll('tr'):
    cells = []
    for cell in row.findAll(['td']):
        text = cell.text
        text = text.strip()
        cells.append(str(text))
    rows.append(cells)


In [3]:
print(rows[0])
headers = rows[0]


['Establishments', 'Address', 'City', 'Zip', 'County', 'Inspection Type', 'Inspection Date', 'Violations']


We now have all the data from the table on the URL inside of an array. Looking at the first element in the array, we can see the headers for the information that we got. We want to treat these headers as separate so that they do not interfere with our values for data. We can read the data from an array into a pandas Data Frame.

In [4]:
df = pd.DataFrame(rows[1:], columns=(rows[0]))
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11344 entries, 0 to 11343
Data columns (total 8 columns):
Establishments     11344 non-null object
Address            11344 non-null object
City               11344 non-null object
Zip                11344 non-null object
County             11344 non-null object
Inspection Type    11344 non-null object
Inspection Date    11344 non-null object
Violations         11344 non-null object
dtypes: object(8)
memory usage: 709.1+ KB


In [5]:
total = 0
for row in df.itertuples():
    if row[8] == 'None':
        total += 1
print(total)



6749


Now that the data is entered into a dataframe in Pandas we can do some basic evaluation of the data. We can go through all of the rows in the dataframe and see if the row with the title `Violations` has the value of 'None'. If it does have the value of None, we can add 1 to a total variable to keep track of the total number of restaurants that have had violations. 

From the above cells, we can see that there are 11,344 total entries in the inspections table on the government website. Therefore of the 11,344 inspections that took place, there were 6,749 that had inspections leaving 4,595 inspections that did not have any violations. This does not equate to 4,595 restaurants not having violations in the past few years, this only accounts for the inspections themselves. In some instances a place could have already had a violation and then for the followup visit there could have been no violations.

Using a modified version of the code above we can check to see which counties the inspections and violations took place.


In [6]:
from collections import defaultdict
d = defaultdict(int)

for row in df.itertuples():
    d[row[5]] += 1
print(d.items())

dict_items([('Kent', 1513), ('New Castle County', 6009), ('Sussex', 3822)])


Since the state of Delaware only has three different counties, this is fairly trivial. We can see that most of the inspections took place in New Castle County which has higher populations than the other counties. More people, more demand for food thus more places to be able to get food.

We can do the same but with zip codes:

In [7]:
from collections import defaultdict
d = defaultdict(int)

for row in df.itertuples():
    d[row[4]] += 1
print(d.items())

dict_items([('19901', 653), ('19720', 522), ('19808', 223), ('19973', 266), ('19801', 1081), ('19930', 204), ('19971', 893), ('19803', 387), ('19709', 619), ('19958', 397), ('19947', 313), ('19810', 191), ('19802', 224), ('19944', 101), ('19805', 478), ('19711', 442), ('19702', 304), ('19701', 413), ('19809', 86), ('19804', 183), ('19713', 319), ('19968', 129), ('19956', 192), ('19963', 259), ('19904', 226), ('19967', 40), ('19977', 178), ('19806', 143), ('19975', 160), ('19966', 445), ('19970', 79), ('19950', 50), ('19952', 134), ('19945', 48), ('10071', 3), ('19934', 110), ('19703', 109), ('09703', 1), ('19850', 4), ('19718', 6), ('19707', 53), ('19946', 21), ('19807', 77), ('19931', 2), ('19939', 38), ('19734', 50), ('19933', 96), ('19951', 25), ('19964', 10), ('19941', 28), ('19730', 16), ('19733', 5), ('19936', 9), ('19980', 13), ('19960', 18), ('19938', 19), ('19969', 3), ('19943', 48), ('19706', 30), ('19705', 2), ('19903', 14), ('19940', 42), ('19880', 3), ('19962', 21), ('1995

In [8]:
sorted(list(d.keys()))

['09703',
 '10030',
 '10058',
 '10071',
 '18838',
 '1970+',
 '19701',
 '19702',
 '19703',
 '19705',
 '19706',
 '19707',
 '19708',
 '19709',
 '19711',
 '19713',
 '19716',
 '19718',
 '19720',
 '19730',
 '19731',
 '19733',
 '19734',
 '19735',
 '19739',
 '19801',
 '19802',
 '19803',
 '19804',
 '19805',
 '19806',
 '19807',
 '19808',
 '19809',
 '19810',
 '19850',
 '19880',
 '19890',
 '1990',
 '19900',
 '19901',
 '19903',
 '19904',
 '1991',
 '19930',
 '19931',
 '19933',
 '19934',
 '19936',
 '19938',
 '19939',
 '19940',
 '19941',
 '19942',
 '19943',
 '19944',
 '19945',
 '19946',
 '19947',
 '19950',
 '19951',
 '19952',
 '19953',
 '19954',
 '19955',
 '19956',
 '19958',
 '19960',
 '19961',
 '19962',
 '19963',
 '19964',
 '19966',
 '19967',
 '19968',
 '19969',
 '19970',
 '19971',
 '19973',
 '19975',
 '19977',
 '19980',
 '21620',
 '`9960']

Looking through our list of sorted Zip Codes we can discover our first mistakes in the dataset. '\`9960' is not a valid zip code as it contains an illegal character. Similarlly `1991` and `1990` are both only 4 numbers long, so those are invalid. `1970+` is another invalid Zip Code. Zip Codes `21620, 10030, 10058, and 18838` are non Delaware and even non US zip codes. However, these can easily be resolved by using the remaining data to correctly update the information.

We can get the total number of unique restaurants from the dataframe with the following:


In [9]:
len(df['Establishments'].unique())


3658

In [37]:
total = 0
data = []
for row in df.itertuples():
    if '8-403.40' in row[8]:
        total += 1
#         print(row)
        data.append(row[1])
print(total)
print(data)


0
[]
